In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt

import TheoreticallyOptimalStrategy as TOS
from marketsimcode import compute_portvals, compute_stats
import indicators
from strategy_evaluation.ManualStrategy import ManualStrategy
from strategy_evaluation.StrategyLearner import StrategyLearner

#STRATEGY LEARNER

In [2]:
symbol = "ML4T-220"
in_sample_sd = dt.datetime(2008, 1, 1)
in_sample_ed = dt.datetime(2009, 12, 31)
out_sample_sd = dt.datetime(2010, 1, 1)
out_sample_ed = dt.datetime(2011, 12, 31)
sv = 100000
commission = 9.95
impact = 0.005

strategylearner1 = StrategyLearner(impact=impact, commission=commission)
strategylearner1.add_evidence(symbol=symbol, sd=in_sample_sd, ed=in_sample_ed)
trades1 = strategylearner1.testPolicy(symbol=symbol, sd=in_sample_sd, ed=in_sample_ed)
portvals1 = compute_portvals(trades1, start_val=sv, commission=commission, impact=impact)
# portvals = compute_portvals(trades, start_val=sv, commission=commission, impact=impact)
print(compute_stats(portvals1))

strategylearner2 = StrategyLearner(impact=impact, commission=commission)
strategylearner2.add_evidence(symbol=symbol, sd=in_sample_sd, ed=in_sample_ed)
trades2 = strategylearner2.testPolicy(symbol=symbol, sd=in_sample_sd, ed=in_sample_ed)
portvals2 = compute_portvals(trades2, start_val=sv, commission=commission, impact=impact)
# portvals = compute_portvals(trades, start_val=sv, commission=commission, impact=impact)
print(compute_stats(portvals2))

strategylearner3 = StrategyLearner(impact=impact, commission=commission)
strategylearner3.add_evidence(symbol=symbol, sd=in_sample_sd, ed=in_sample_ed)
trades3 = strategylearner3.testPolicy(symbol=symbol, sd=in_sample_sd, ed=in_sample_ed)
portvals3 = compute_portvals(trades3, start_val=sv, commission=commission, impact=impact)
# portvals = compute_portvals(trades, start_val=sv, commission=commission, impact=impact)
print(compute_stats(portvals3))

q1-table coverage:  0.5185185185185185
            ML4T-220
2008-01-02         2
2008-01-03         2
2008-01-04         2
2008-01-07         2
2008-01-08         2
...              ...
2009-12-24         0
2009-12-28         1
2009-12-29         0
2009-12-30         0
2009-12-31         0

[505 rows x 1 columns]
(-3.8359653388691948, -0.0012277264285571869, 0.28413741329965536, -0.06859199787023115)
q1-table coverage:  0.5308641975308642
            ML4T-220
2008-01-02         1
2008-01-03         1
2008-01-04         1
2008-01-07         1
2008-01-08         1
...              ...
2009-12-24         0
2009-12-28         0
2009-12-29         0
2009-12-30         0
2009-12-31         0

[505 rows x 1 columns]
(5.702374292139204, 0.0038074092828533244, 0.008130860333017662, 7.4334997940839385)
q1-table coverage:  0.5308641975308642
            ML4T-220
2008-01-02         1
2008-01-03         1
2008-01-04         1
2008-01-07         1
2008-01-08         1
...              ...
2009-12-24

#MANUAL STRATEGY

In [20]:
# Manual Strategy

symbol = "JPM"
in_sample_sd = dt.datetime(2008, 1, 1)
in_sample_ed = dt.datetime(2009, 12, 31)
out_sample_sd = dt.datetime(2010, 1, 1)
out_sample_ed = dt.datetime(2011, 12, 31)
sv = 100000
commission = 9.95
impact = 0.005

# sd = out_sample_sd
# ed = out_sample_ed

sd = in_sample_sd
ed = in_sample_ed

indicator = indicators.Indicators(symbol, pd.date_range(sd, ed))
bbp = indicator.get_bbp(window = 20)
rsi = indicator.get_rsi(window = 14)
macd = indicator.get_macd().loc[sd:] # My version of indicators.py is also returning values 9 market days prior to sd
ppo = indicator.get_ppo()

# Calculate MACD crossover
macd_prev = macd.shift(1)
macd_prev.iloc[0] = macd_prev.iloc[1]
conditions = [
    (macd_prev <= 0) & (macd > 0),
    (macd_prev >= 0) & (macd < 0)
]
values = [1, -1]
# Documentation: https://numpy.org/doc/2.1/reference/generated/numpy.select.html
macd_crossover = pd.DataFrame({'result': np.select(conditions, values, default=0)}, index=macd.index, columns=["result"])

# Generate signals
# signals_cond = [
#     ((bbp <= 0) & (rsi <= 30)) | ((bbp <= 0) & (macd_crossover["result"] == 1)) | ((rsi <= 30) & (macd_crossover["result"] == 1)),
#     ((bbp >= 100) & (rsi >= 70)) | ((bbp >= 100) & (macd_crossover["result"] == 1)) | ((rsi >= 70) & (macd_crossover["result"] == 1))
# ]

signals_cond = [
            ((bbp <= 0) & (rsi <= 30)) | ((bbp <= 0) & (ppo > 1)) | ((rsi <= 30) & (ppo > 1)),
            ((bbp >= 100) & (rsi >= 70)) | ((bbp >= 100) & (ppo < -1)) | ((rsi >= 70) & (ppo < -1))
        ]
signal_values = [1, -1]
signals = pd.DataFrame({'result': np.select(signals_cond, signal_values, default=0)}, index=bbp.index)

# Based on indicators on T day, we will place orders on T+1 day
signals = signals.shift(1)
signals = signals.fillna(0)
signals.drop(signals.index[-1])

# Generate Manual Strategy trades
net_position = 0
manual_trades = pd.DataFrame(0, index=bbp.index, columns=[symbol])

for date in signals.index:
    if signals.loc[date].iloc[0] == 1:
        manual_trades.loc[date] = 1000 - net_position
    elif signals.loc[date].iloc[0] == -1:
        manual_trades.loc[date] = -1000 - net_position

    net_position += manual_trades.loc[pd.to_datetime(date), symbol]

# portvals = compute_portvals(manual_trades, start_val=sv, commission=commission, impact=impact)
portvals = compute_portvals(manual_trades, start_val=sv, commission=commission, impact=impact)

print(compute_stats(portvals))

# manual_trades
ppo

(0.12344300000000086, 0.00026066868511829747, 0.007720292661427542, 0.5359883716721906)


2010-01-04    0.177419
2010-01-05    0.741681
2010-01-06    1.214082
2010-01-07    1.800100
2010-01-08    2.171187
                ...   
2011-12-23    1.184968
2011-12-27    1.313471
2011-12-28    1.241590
2011-12-29    1.466797
2011-12-30    1.547238
Name: JPM, Length: 504, dtype: float64

In [36]:
compute_stats(portvals)

(-0.570665, -0.0016337761780937103, 0.05524187128897186, -0.46948794792624565)

In [8]:
symbol = "ML4T-220"
in_sample_sd = dt.datetime(2008, 1, 1)
in_sample_ed = dt.datetime(2009, 12, 31)
out_sample_sd = dt.datetime(2010, 1, 1)
out_sample_ed = dt.datetime(2011, 12, 31)
sv = 100000
commission = 9.95
impact = 0.005

# sd = out_sample_sd
# ed = out_sample_ed

sd = in_sample_sd
ed = in_sample_ed

indicator = indicators.Indicators(symbol, pd.date_range(sd, ed))
bbp = indicator.get_bbp(window = 20)
rsi = indicator.get_rsi(window = 14)
macd = indicator.get_macd().loc[sd:] # My version of indicators.py is also returning values 9 market days prior to sd
ppo = indicator.get_ppo()

ppo

2008-01-02   -1.633581
2008-01-03   -1.749126
2008-01-04   -1.784952
2008-01-07   -1.737765
2008-01-08   -1.608489
                ...   
2009-12-24    0.416114
2009-12-28    0.024218
2009-12-29   -0.370356
2009-12-30   -0.750578
2009-12-31   -1.099777
Name: ML4T-220, Length: 505, dtype: float64